# Portfolio Analysis

In this project, we seek to run through some classic and modern portfolio construction and asset allocation strategies like Modern Portfolio Theory (Markovitz, 1952), Black-Litterman Model (Black & Litterman, 1992) , Hierarchical Risk Parity (Lopez de Prado, 2016) etc. to gain some insights into the usefulness of standard risk measures like sharpe ratio and maximum drawdown in constructing a risk-adjusted portfolio. The end goal is to test such strategies to a sufficient level of rigour that we can convincingly employ them into our portfolio rebalancing pipeline. I will start off by testing my own portfolio and the procedures should be generalizable. Let's get started! 

(**Note**: *Some of the inputs used are based off my personal fundamental analysis so they are not yet documented.*)

## 0. Project Setup

##### Assets

1. Sea Limited (NYSE:'**SE**')
2. Unity Software Inc (NYSE:'**U**')
3. NVIDIA Corporation (NASDAQ:'**NVDA**') 
4. Bilibili Inc (NASDAQ:'**BILI**')
5. Facebook Inc (NASDAQ:'**FB**')
6. Draftkings Inc (NASDAQ:'**DKNG**')
7. Pinduoduo Inc (NASDAQ:'**PDD**')
8. Alphabet Inc (NASDAQ:'**GOOG**)
9. Amazon.com Inc (NASDAQ:'**AMZN**)
10. Roblox Corp (NYSE:'**RBLX**')
11. Clearpoint Neuro Inc(NASDAQ: '**CLPT**')
12. Intellia Therapeutics Inc (NASDAQ: '**NTLA**')
13. Coinbase Global Inc (NASDAQ: '**COIN**')
14. Upstart Holdings Inc (NASDAQ: '**UPST**')

##### Data

Daily adjusted closing prices from yahoo finance API 

##### Time period

1 year, 3 year, 5 year & 10 year.

In [3]:
# Load the required packages 
# Computation
import numpy as np 
from scipy import fftpack
# Plotting
import matplotlib.pyplot as plt
import mplcursors
import matplotlib.ticker as mtick
import seaborn as sns
# Data analysis
import pandas as pd
from sklearn import preprocessing
# Data source
import yfinance as yf

## 1. Data Analysis

In [4]:
# shortlisted stocks for portfolio analysis
s_list = 'SE U NVDA BILI FB DKNG PDD GOOG AMZN RBLX NTLA COIN UPST CLPT'
df_1y = yf.download(tickers = s_list, period = '1y', interval = '1d', group_by = 'ticker')
df_3y = yf.download(tickers = s_list, period = '3y', interval = '1d', group_by = 'ticker')
df_5y = yf.download(tickers = s_list, period = '5y', interval = '1d', group_by = 'ticker')
df_10y = yf.download(tickers = s_list, period = '10y', interval = '1d', group_by = 'ticker')

# Benchmark - S&P500
bench = '^GSPC'
bench_1y = yf.download(tickers = bench, period = '1y', interval = '1d')
bench_3y = yf.download(tickers = bench, period = '3y', interval = '1d')
bench_5y = yf.download(tickers = bench, period = '5y', interval = '1d')
bench_10y = yf.download(tickers = bench, period = '10y', interval = '1d')

[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  14 of 14 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:
# select column for adjusted close prices
df_close_1y = df_1y.xs('Adj Close', level = 1, axis = 1)
df_close_3y = df_3y.xs('Adj Close', level = 1, axis = 1)
df_close_5y = df_5y.xs('Adj Close', level = 1, axis = 1)
df_close_10y = df_10y.xs('Adj Close', level = 1, axis = 1)
bench_close_1y = pd.DataFrame(data = bench_1y['Adj Close'], columns = ['Adj Close'])
bench_close_3y = pd.DataFrame(data = bench_3y['Adj Close'], columns = ['Adj Close'])
bench_close_5y = pd.DataFrame(data = bench_5y['Adj Close'], columns = ['Adj Close'])
bench_close_10y = pd.DataFrame(data = bench_10y['Adj Close'], columns = ['Adj Close'])

# adding column name to benchmark table
bench_close_1y = bench_close_1y.rename(columns = {'Adj Close':'S&P500'})
bench_close_3y = bench_close_3y.rename(columns = {'Adj Close':'S&P500'})
bench_close_5y = bench_close_5y.rename(columns = {'Adj Close':'S&P500'})
bench_close_10y = bench_close_10y.rename(columns = {'Adj Close':'S&P500'})
 
# reordering column names in portfolio 
s_order = ['SE', 'U', 'NVDA', 'BILI', 'FB', 'DKNG', 'PDD', 'GOOG', 'AMZN', 'RBLX', 'NTLA', 'COIN', 'UPST', 'CLPT']
df_close_1y = df_close_1y[s_order]
df_close_3y = df_close_3y[s_order]
df_close_5y = df_close_5y[s_order]
df_close_10y = df_close_10y[s_order]

# appending benchmark prices to portfolio dataframe 
df_close_1y = pd.concat([df_close_1y,bench_close_1y], axis = 1)
df_close_3y = pd.concat([df_close_3y,bench_close_3y], axis = 1)
df_close_5y = pd.concat([df_close_5y,bench_close_5y], axis = 1)
df_close_10y = pd.concat([df_close_10y,bench_close_10y], axis = 1)

# drop row if all values are NaN
df_close_1y.dropna(axis = 0, how = 'all', inplace = True)
df_close_3y.dropna(axis = 0, how = 'all', inplace = True)
df_close_5y.dropna(axis = 0, how = 'all', inplace = True)
df_close_10y.dropna(axis = 0, how = 'all', inplace = True)

# check the last 5 trading days
df_close_1y.tail(5)

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT,S&P500
Date,,,,,,,,,,,,,,,
2021-10-05,320.940002,128.050003,204.509995,64.500000,332.959991,49.290001,87.419998,2723.540039,3221.000000,72.570000,129.759995,240.089996,296.299988,17.150000,4345.720215
2021-10-06,315.339996,130.380005,207.000000,64.269997,333.640015,48.990002,89.320000,2747.080078,3262.010010,73.699997,129.690002,250.380005,307.700012,17.280001,4363.549805
2021-10-07,324.250000,136.070007,210.750000,70.400002,329.220001,49.369999,94.949997,2783.709961,3302.429932,74.800003,134.779999,251.589996,307.380005,17.250000,4399.759766
2021-10-08,323.279999,136.220001,208.309998,70.959999,330.049988,47.910000,96.220001,2801.120117,3288.620117,70.440002,120.709999,248.139999,311.230011,17.049999,4391.339844
2021-10-11,323.350006,135.580002,208.343506,71.849998,328.375000,49.470001,97.925003,2798.857422,3279.794922,70.949997,122.529999,252.580002,307.399994,17.043600,4394.120117


In [6]:
# normalize the prices 
n = len(df_close_1y.columns)
df_close_1y_norm = df_close_1y.copy()
df_close_3y_norm = df_close_3y.copy()
df_close_5y_norm = df_close_5y.copy()
df_close_10y_norm = df_close_10y.copy()

for i in range(n): 
    a = df_close_1y.iloc[:,i].first_valid_index()
    df_close_1y_start = df_close_1y.iloc[:,i].loc[a]
    df_close_1y_norm.iloc[:,i] = (df_close_1y.iloc[:,i] - df_close_1y_start)/df_close_1y_start
    
    b = df_close_3y.iloc[:,i].first_valid_index()
    df_close_3y_start = df_close_3y.iloc[:,i].loc[b]
    df_close_3y_norm.iloc[:,i] = (df_close_3y.iloc[:,i] - df_close_3y_start)/df_close_3y_start
    
    c = df_close_5y.iloc[:,i].first_valid_index()
    df_close_5y_start = df_close_5y.iloc[:,i].loc[c]
    df_close_5y_norm.iloc[:,i] = (df_close_5y.iloc[:,i] - df_close_5y_start)/df_close_5y_start
    
    d = df_close_10y.iloc[:,i].first_valid_index()
    df_close_10y_start = df_close_10y.iloc[:,i].loc[d]
    df_close_10y_norm.iloc[:,i] = (df_close_10y.iloc[:,i] - df_close_10y_start)/df_close_10y_start

In [50]:
# plotting out the prices
%matplotlib widget

# plot configurations
sns.set(style="darkgrid", font_scale=0.8)
palette = sns.color_palette("hls", n)
fig1, ax1 = plt.subplots(figsize=(8, 4))

# plotting out the figure
plot1 = sns.lineplot(ax=ax1, data = df_close_1y_norm, dashes = False, palette=palette)
plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0., fontsize = 8)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.title('1 Year Returns')

# make it interactive
cursor = mplcursors.cursor(plot1, hover=True)
@cursor.connect("add")
def on_add(sel):
    sel.annotation.set(text=tt[sel.target.index])
    
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# convert the price into rate of return
df_return_1y = df_close_1y.pct_change()
df_return_3y = df_close_3y.pct_change()
df_return_5y = df_close_5y.pct_change()
df_return_10y = df_close_10y.pct_change()

# check the descriptive statistics of 1 year rate of return
df_return_1y.describe()

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT,S&P500
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,149.000000,251.000000,125.000000,205.000000,251.000000,251.000000
mean,0.003242,0.002389,0.001838,0.002772,0.000889,0.000509,0.001664,0.002426,-0.000065,0.001015,0.008236,-0.001481,0.015410,0.005816,0.000904
std,0.034940,0.038857,0.025050,0.048401,0.019693,0.034813,0.047440,0.015434,0.016024,0.042416,0.060898,0.035416,0.096743,0.048992,0.008512
min,-0.103195,-0.141312,-0.082178,-0.170397,-0.063099,-0.084906,-0.122561,-0.054630,-0.075649,-0.123584,-0.107134,-0.078664,-0.281373,-0.165829,-0.035288
25%,-0.015658,-0.022634,-0.011683,-0.029688,-0.010715,-0.022796,-0.027439,-0.005270,-0.008459,-0.027302,-0.028986,-0.024695,-0.039428,-0.021156,-0.003453
50%,0.003474,0.002277,0.002297,0.001371,-0.000224,-0.001582,-0.001753,0.002454,0.000719,0.000529,0.003632,-0.006277,0.006190,0.000345,0.001015
75%,0.025901,0.025932,0.016029,0.032242,0.013024,0.019598,0.021967,0.009114,0.009458,0.026530,0.035809,0.016587,0.047685,0.031471,0.006018
max,0.108809,0.132524,0.080333,0.221735,0.083227,0.116854,0.222496,0.073961,0.063230,0.213281,0.502083,0.112818,0.893239,0.157664,0.023791


### Rate of Return

In [9]:
# compute the annualized average rate of return across a 1 year, 3 year, 5 year & 10 year horizon.
nday = 252 # number of trading days
df_return_1y_mean = pd.DataFrame(data = df_return_1y.mean(axis = 0), columns = ['1Y Annual Return']) * nday
df_return_3y_mean = pd.DataFrame(data = df_return_3y.mean(axis = 0), columns = ['3Y Annual Return']) * nday
df_return_5y_mean = pd.DataFrame(data = df_return_5y.mean(axis = 0), columns = ['5Y Annual Return']) * nday
df_return_10y_mean = pd.DataFrame(data = df_return_10y.mean(axis = 0), columns = ['10Y Annual Return']) * nday
data_frames = [df_return_1y_mean, df_return_3y_mean, df_return_5y_mean, df_return_10y_mean]
df_return_merge = pd.concat(data_frames, axis = 1)

#df_return_10y_mean.plot.barh()
df_return_merge

,1Y Annual Return,3Y Annual Return,5Y Annual Return,10Y Annual Return
SE,0.816935,1.274988,0.923959,0.923959
U,0.601936,0.858655,0.858655,0.858655
NVDA,0.463203,0.554378,0.626346,0.495256
BILI,0.698560,0.777328,0.736814,0.736814
FB,0.223921,0.320465,0.242065,0.296362
DKNG,0.128214,0.932357,0.932274,0.932274
PDD,0.419301,0.737922,0.658163,0.658163
GOOG,0.611384,0.365153,0.291262,0.265864
AMZN,-0.016371,0.268863,0.318347,0.311164
RBLX,0.255888,0.258890,0.258890,0.258890


We see that the annualized returns are skewed to the right due to the broad spike in equity valuations in 2020. Also, some of the companies in the list only went public in the last 5 years, which explains the potentially transitory outperformance against the benchmark. A subtle takeaway is that some of the returns are largely driven by a particular breakout year (e.g. AMZN from March to September 2020), which serves as a timely caution against making large portfolio adjustments based on 1-year performance alone. 

### Variance

In [10]:
# compute the daily standard deviation across a 1 year, 3 year, 5 year & 10 year horizon.
df_std_1y = pd.DataFrame(data = df_close_1y_norm.std(axis = 0), columns = ['1Y Daily Standard Deviation'])  
df_std_3y = pd.DataFrame(data = df_close_3y_norm.std(axis = 0), columns = ['3Y Daily Standard Deviation']) 
df_std_5y = pd.DataFrame(data = df_close_5y_norm.std(axis = 0), columns = ['5Y Daily Standard Deviation']) 
df_std_10y = pd.DataFrame(data = df_close_10y_norm.std(axis = 0), columns = ['10Y Daily Standard Deviation']) 
data_frames = [df_std_1y, df_std_3y, df_std_5y, df_std_10y]
df_std_merge = pd.concat(data_frames, axis = 1)

# compute the daily variance across a 1 year, 3 year, 5 year & 10 year horizon.
df_var_1y = pd.DataFrame(data = df_close_1y_norm.var(axis = 0), columns = ['1Y Daily Variance'])  
df_var_3y = pd.DataFrame(data = df_close_3y_norm.var(axis = 0), columns = ['3Y Daily Variance']) 
df_var_5y = pd.DataFrame(data = df_close_5y_norm.var(axis = 0, skipna = True), columns = ['5Y Daily Variance']) 
df_var_10y = pd.DataFrame(data = df_close_10y_norm.var(axis = 0, skipna = True), columns = ['10Y Daily Variance']) 
data_frames = [df_var_1y, df_var_3y, df_var_5y, df_var_10y]
df_var_merge = pd.concat(data_frames, axis = 1)

df_var_merge

,1Y Daily Variance,3Y Daily Variance,5Y Daily Variance,10Y Daily Variance
SE,0.092300,75.458489,37.833189,37.833189
U,0.056179,0.103669,0.103669,0.103669
NVDA,0.054358,0.910997,9.638293,220.818521
BILI,0.322312,8.956915,10.830379,10.830379
FB,0.019543,0.183500,0.239984,4.943467
DKNG,0.021187,3.630957,3.630953,3.630953
PDD,0.140604,5.576119,3.366872,3.366872
GOOG,0.069467,0.247045,0.436196,4.363891
AMZN,0.002130,0.174885,1.086916,18.365290
RBLX,0.012993,0.012990,0.012990,0.012990


In [11]:
# compute the pearson pairwise correlation matrix
df_return_1y_corr = df_close_1y_norm.corr(method='pearson')
df_return_3y_corr = df_close_3y_norm.corr(method='pearson')
df_return_5y_corr = df_close_5y_norm.corr(method='pearson')
df_return_10y_corr = df_close_10y_norm.corr(method='pearson')

In [51]:
# plotting the 1 year correlation matrix 

sns.set_theme(style="white")
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(df_return_1y_corr, dtype=bool))

# set up the matplotlib figure
fig2, ax2 = plt.subplots(figsize=(11, 9))

# generate a custom diverging colormap
cmap = sns.diverging_palette(240, 5, as_cmap=True)

# draw the heatmap with the mask and correct aspect ratio
sns.heatmap(df_return_1y_corr, mask=mask, cmap=cmap, vmax=1, center=0, vmin = -1,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

This result is sort of expected. In general, we know that the big tech should be highly correlated with the S&P500, and the so-called "high-beta/ARK" stocks should correlate with each other.

In [13]:
# compute the pairwise variance covariance matrix 
df_return_1y_cov = df_close_1y_norm.cov() 
df_return_3y_cov = df_close_3y_norm.cov()
df_return_5y_cov = df_close_5y_norm.cov()
df_return_10y_cov = df_close_10y_norm.cov()

### Risk-Return Measures

In [14]:
# Sharpe ratio 
df_sharpe_merge = np.divide(df_return_merge,df_std_merge)
df_sharpe_merge.rename(columns = {'1Y Annual Return':'1Y Sharpe', '3Y Annual Return':'3Y Sharpe', '5Y Annual Return':'5Y Sharpe','10Y Annual Return':'10Y Sharpe'}, inplace=True)
df_sharpe_merge

,1Y Sharpe,3Y Sharpe,5Y Sharpe,10Y Sharpe
SE,2.688967,0.146775,0.150216,0.150216
U,2.539588,2.666818,2.666818,2.666818
NVDA,1.986740,0.580828,0.201750,0.033328
BILI,1.230456,0.259732,0.223891,0.223891
FB,1.601749,0.748106,0.494129,0.133293
DKNG,0.880842,0.489296,0.489253,0.489253
PDD,1.118218,0.312496,0.358691,0.358691
GOOG,2.319659,0.734660,0.441005,0.127269
AMZN,-0.354738,0.642916,0.305354,0.072609
RBLX,2.244885,2.271477,2.271477,2.271477


Comparing the sharpe ratios of the individual stocks against the benchmark, it becomes apparent that beating the market on a risk-adjusted basis is no simple feat (at least when we use volatility as a risk measure). The sharpe ratio of individual names might be higher than the S&P500 on their peak years, but they tend to fall behind on historical aggregates of longer time horizons.

## 2. Portfolio Optimization

In this section, we will start off by experimenting with the PyPortfolioOpt library (Martin, 2021). His work can be accessed [here](https://joss.theoj.org/papers/10.21105/joss.03066).

In [15]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.hierarchical_portfolio import HRPOpt

### Current Allocation

In [16]:
# current weightage 
arr_w = np.array([[0.40,0.12,0.10,0.06,0.06,0.06,0.04,0.04,0.04,0.04,0.01, 0.01, 0.01, 0.01]])
df_w = pd.DataFrame(data = arr_w, columns = s_order)
df_w = df_w.rename(index={0: "weight"})
df_w

,SE,U,NVDA,BILI,FB,DKNG,PDD,GOOG,AMZN,RBLX,NTLA,COIN,UPST,CLPT
weight,0.4,0.12,0.1,0.06,0.06,0.06,0.04,0.04,0.04,0.04,0.01,0.01,0.01,0.01


In [85]:
fig3, ax3 = plt.subplots()
ax3.pie(df_w.loc['weight'], labels = s_order,normalize=False);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# split dataframe into portfolio and benchmark
df_return_1y_mean_port = df_return_1y_mean.iloc[0:n-1,:]
df_return_1y_mean_bench = df_return_1y_mean.loc['S&P500']

df_return_1y_cov_port = df_return_1y_cov.iloc[0:n-1,0:n-1]

# convert dataframe back to numpy array for ease of manipulation
np_return_1y_mean = df_return_1y_mean_port.T.to_numpy()
np_return_1y_cov = df_return_1y_cov_port.to_numpy()

In [113]:
# portfolio return 
portfolio_return_1y = np.inner(np_return_1y_mean,arr_w)
portfolio_var_1y = np.matmul(np.matmul(arr_w,np_return_1y_cov),arr_w.T)
portfolio_vol_1y = np.sqrt(portfolio_var_1y)
portfolio_sharpe_1y = portfolio_return_1y/np.sqrt(portfolio_var_1y)
benchmark_sharpe_1y = df_sharpe_merge['1Y Sharpe']['S&P500']
print(f'Portfolio 1y return:{portfolio_return_1y}; Portfolio 1y variance:{portfolio_var_1y}; Portfolio 1y volatility:{portfolio_vol_1y}')
print(f'Portfolio 1y sharpe ratio:{portfolio_sharpe_1y}; S&P500 1y Sharpe: {benchmark_sharpe_1y}')

Portfolio 1y return:[[0.62968786]]; Portfolio 1y variance:[[0.04521599]]; Portfolio 1y volatility:[[0.21264053]]
Portfolio 1y sharpe ratio:[[2.9612787]]; S&P500 1y Sharpe: 2.4391930105070707


Here, we see the diversification effects of a multi-asset portfolio. While the 1 year portfolio returns remains at a respectable 68%, the portfolio volatility is drastically reduced as compared to the volatility of the individual stocks. In MPT terminology, the idiosyncratic risks of individual assets is reduced.

However, we once again see the power of the S&P500, which boosts a Sharpe ratio of 2.60 compared to our current portfolio Sharpe ratio of 2.87. Now let's see if we can use the MPT techniques to construct a portfolio that outperforms the market on a risk-adjusted basis. 

### Maximum Sharpe Portfolio 

In [22]:
# split dataframe into portfolio and benchmark
df_close_1y_port = df_close_1y.iloc[:,0:n-1]
df_close_1y_bench = df_close_1y['S&P500']

In [23]:
# Expected returns
mu_a = expected_returns.mean_historical_return(df_close_1y_port, compounding = False)
mu_g = expected_returns.mean_historical_return(df_close_1y_port, compounding = True)
mu_ema = expected_returns.ema_historical_return(df_close_1y_port, compounding=True)
mu_capm = expected_returns.capm_return(df_close_1y_port)
data_frames = [mu_a,mu_g,mu_ema,mu_capm]
df_merge = pd.concat(data_frames, axis = 1)
df_merge = df_merge.rename(columns = {0: 'Arithmetic', 1: 'Geometric', 2:'Exponential MA', 'mkt': 'CAPM'})
df_merge

,Arithmetic,Geometric,2021-10-11 00:00:00,CAPM
SE,0.816935,0.939233,1.140653,1.316552
U,0.601936,0.508906,0.786875,1.376036
NVDA,0.463203,0.468295,0.689609,0.831161
BILI,0.698560,0.503055,0.326789,1.737116
FB,0.223921,0.191672,0.203006,0.522149
DKNG,0.128214,-0.022420,0.061948,0.999018
PDD,0.419301,0.156534,0.148183,1.631149
GOOG,0.611384,0.787794,0.738856,0.362772
AMZN,-0.016371,-0.047567,-0.007078,0.450188
RBLX,0.255888,0.035539,0.117167,1.113664


In [24]:
# Risk matrix
S = risk_models.sample_cov(df_close_1y_port)

In [89]:
# Optimize for maximum Sharpe ratio using arithmetic mean & sample covariance matrix
ef_a = EfficientFrontier(mu_a, S)
raw_weights1 = ef_a.max_sharpe()
cleaned_weights1 = ef_a.clean_weights()
# ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights1)
ef_a.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.0), ('U', 0.0), ('NVDA', 0.0), ('BILI', 0.0), ('FB', 0.0), ('DKNG', 0.0), ('PDD', 0.0), ('GOOG', 0.69652), ('AMZN', 0.0), ('RBLX', 0.0), ('NTLA', 0.10335), ('COIN', 0.0), ('UPST', 0.11804), ('CLPT', 0.08209)])
Expected annual return: 121.9%
Annual volatility: 30.9%
Sharpe Ratio: 3.89


(1.2190373245903872, 0.30857391536065754, 3.885737792155784)

In [103]:
fig4, ax4 = plt.subplots()
ax4.pie(pd.Series(cleaned_weights1), labels = cleaned_weights1.keys())
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Here, we see the central limitation behind the methodology of using historical returns to estimate the expected (future) returns. The allocation model placed a disproportionate weight on the outperformers in the past year (i.e. google, upstart & clearpoint), outputing a portfolio that looks good on paper but probably does not hold its weight over time. Let's try to do better. 

In [91]:
# Optimize for maximum Sharpe ratio using capm returns & sample covariance matrix
ef_capm = EfficientFrontier(mu_capm, S)
raw_weights2 = ef_capm.max_sharpe()
cleaned_weights2 = ef_capm.clean_weights()
print(cleaned_weights2)
ef_capm.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.07485), ('U', 0.06849), ('NVDA', 0.05843), ('BILI', 0.06903), ('FB', 0.06377), ('DKNG', 0.06637), ('PDD', 0.07125), ('GOOG', 0.08472), ('AMZN', 0.10912), ('RBLX', 0.05868), ('NTLA', 0.06246), ('COIN', 0.06475), ('UPST', 0.07211), ('CLPT', 0.07598)])
Expected annual return: 116.6%
Annual volatility: 34.4%
Sharpe Ratio: 3.34


(1.1656148054080397, 0.34350221390786967, 3.3351016646294562)

In [102]:
fig5, ax5 = plt.subplots()
ax5.pie(pd.Series(cleaned_weights2), labels = cleaned_weights2.keys())
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Interestingly, we see that using $\mu_{CAPM}$ gives us a portfolio allocation that is almost equal weighting across all assets. However, it's absolute and risk-adjusted returns are both inferior compared to the arithmetic mean portfolio. 

### Black-Litterman Portfolio

In [105]:
S = risk_models.sample_cov(df_close_1y_port)
view_dict = {"SE": 0.50, "NVDA": 0.40, "U": 0.40, "BILI": 0.39, "PDD": 0.37,  "DKNG": 0.35, "FB": 0.33, "GOOG": 0.31, "AMZN": 0.31, "RBLX": 0.27, }
confidence_dict = np.array([0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8])
bl = BlackLittermanModel(S, pi="equal", absolute_views=view_dict, omega="idzorek", view_confidences = confidence_dict,tau=0.1)
rets = bl.bl_returns()
var = pd.DataFrame(np.diag(S), index= S.index , columns = ['Variance'], dtype = float)

# black litterman returns
df_merge = pd.concat([rets, var], axis = 1)
df_merge = df_merge.rename(columns = {0: 'Return'})
df_merge['Sharpe'] = df_merge['Return']/df_merge['Variance']
df_merge

,Return,Variance,Sharpe
SE,0.461252,0.307650,1.499275
U,0.391523,0.380489,1.028999
NVDA,0.383731,0.158132,2.426645
BILI,0.384294,0.590354,0.650955
FB,0.318097,0.097732,3.254780
DKNG,0.334242,0.305418,1.094374
PDD,0.390290,0.567142,0.688170
GOOG,0.283472,0.060030,4.722172
AMZN,0.291718,0.064702,4.508620
RBLX,0.274834,0.453381,0.606187


In [106]:
ef_bl = EfficientFrontier(rets, S)
raw_weights3 = ef_bl.max_sharpe()
cleaned_weights3 = ef_bl.clean_weights()
print(cleaned_weights3)
ef_bl.portfolio_performance(verbose=True)

OrderedDict([('SE', 0.0664), ('U', 0.00874), ('NVDA', 0.06224), ('BILI', 0.0), ('FB', 0.07048), ('DKNG', 0.04039), ('PDD', 0.0), ('GOOG', 0.41605), ('AMZN', 0.30295), ('RBLX', 0.0), ('NTLA', 0.0), ('COIN', 0.02181), ('UPST', 0.00503), ('CLPT', 0.00592)])
Expected annual return: 30.6%
Annual volatility: 22.5%
Sharpe Ratio: 1.27


(0.3063111521581045, 0.22486016530658282, 1.273285340548145)

In [107]:
fig6, ax6 = plt.subplots()
ax6.pie(pd.Series(cleaned_weights3), labels = cleaned_weights3.keys())
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We see that the model still favors stocks with high sharpe-ratio (e.g. GOOG & AMZN).

Now let's try out recent developments in the field like shrinkage and Hierarchical Risk Parity (López de Prado, 2016), along with some novel experimental features like exponentially-weighted covariance matrices.

In [108]:
rets = expected_returns.returns_from_prices(df_close_1y_port)

In [111]:
hrp = HRPOpt(rets)
hrp.optimize()
cleaned_weights4 = hrp.clean_weights()
print(cleaned_weights4)
hrp.portfolio_performance(verbose=True);

OrderedDict([('SE', 0.04566), ('U', 0.03942), ('NVDA', 0.11424), ('BILI', 0.01939), ('FB', 0.11488), ('DKNG', 0.05047), ('PDD', 0.02019), ('GOOG', 0.18703), ('AMZN', 0.21708), ('RBLX', 0.03308), ('NTLA', 0.01649), ('COIN', 0.08568), ('UPST', 0.01161), ('CLPT', 0.04478)])
Expected annual return: 40.0%
Annual volatility: 24.9%
Sharpe Ratio: 1.53


In [112]:
fig7, ax7 = plt.subplots()
ax7.pie(pd.Series(cleaned_weights4), labels = cleaned_weights4.keys(), normalize=False);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Using the HRP optimization model, we see an allocation that is very similar to that of the Black Litterman model, but with slightly better expected returns and sharpe ratio. However, when we compare the Sharpe ratios to that given by the mean-variance optimization models, we see that there is a stark difference. I need to compute the out-of-sample performance of the models to get a clearer picture.

## 3. Out-of-Sample Performance

## 4.Extensions

As per PyPortfolioOpt's roadmap, I aim to include conditional drawdown optimization (Chekhlov et al., 2005), other risk parity optimizations (Spinu,2013), higher moment optimization (Harvey et al., 2010), and factor models into my analysis in the near future. Stay tuned!